In [1]:
#Ellenőrizzuk mi van a taxis adatbázisban
#Exploratory Data Analysis (Feltáró adatelemzés)
#CTRL nyomva tartva -> GoToDefinition
import pandas as pd
import requests
import os
from dateutil.relativedelta import relativedelta
from datetime import datetime as dt

pd.set_option('display.max_columns',30)

data_out = r'../data/'


In [2]:
api_token = os.environ.get('CHICAGO_API_TOKEN')
headers = {"X-App-Token": api_token}
startDate = dt.now().strftime("%Y-%m-%d")
endDate = (dt.now() - relativedelta(months=2)).strftime("%Y-%m-%d")


startDate = dt(2023,9,8).strftime("%Y-%m-%d")

#url = f"https://data.cityofchicago.org/resource/wrvz-psew.json?$where=trip_start_timestamp >= '{startDate}T00:00:00' AND trip_start_timestamp <= '{startDate}T23:59:59'&$limit=30000&$$app_token={api_token}"
#url = f"https://data.cityofchicago.org/resource/wrvz-psew.json?$where=trip_start_timestamp >= '{startDate}T00:00:00' AND trip_start_timestamp <= '{startDate}T23:59:59'&$limit=30000"

url = f"https://data.cityofchicago.org/resource/wrvz-psew.json"

params = f"$where=trip_start_timestamp >= '{startDate}T00:00:00' AND trip_start_timestamp <= '{startDate}T23:59:59'&$limit=30000" # ? nem kell, ha így adod meg
response = requests.get(url, params=params, headers=headers)
data = response.json()
response

<Response [200]>

In [3]:
#Ezzel lehet megnézni, hogy mi a tipusa egy vátozónak, vagy egy obj hova tartozik
type(response)

requests.models.Response

In [4]:
len(data)

20799

In [5]:
taxi_trips = pd.DataFrame(data)


In [6]:
taxi_trips.info()
#Itt lehet azokat az oszlopokat kiszűrni, ahol null van.
#Ellenőrizni a duplikációt, mint pl a koordinátáknál

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20799 entries, 0 to 20798
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   trip_id                     20799 non-null  object
 1   taxi_id                     20799 non-null  object
 2   trip_start_timestamp        20799 non-null  object
 3   trip_end_timestamp          20799 non-null  object
 4   trip_seconds                20793 non-null  object
 5   trip_miles                  20799 non-null  object
 6   pickup_community_area       20239 non-null  object
 7   dropoff_community_area      18863 non-null  object
 8   fare                        20749 non-null  object
 9   tips                        20749 non-null  object
 10  tolls                       20749 non-null  object
 11  extras                      20749 non-null  object
 12  trip_total                  20749 non-null  object
 13  payment_type                20799 non-null  ob

In [7]:
taxi_trips.describe()
#A mapping táblák létrehozásához segítség. Ha sok kis unique érték van, akkor célszerű mapping táblát csinálni, hogy ne foglaljon annyi helyet.

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid_location,pickup_census_tract,dropoff_census_tract
count,20799,20799,20799,20799,20793,20799,20239,18863,20749,20749,20749,20749,20749,20799,20799,20252,20252,20252,19088,19088,19088,9928,9644
unique,20799,2439,96,114,3579,2386,77,77,1383,1088,22,186,2497,7,32,149,149,149,175,175,175,72,103
top,244661871466bd53adbd7e28c6fb95e285ac3511,2780ead18beaa862cc67315ddabd9d1acaadcd6da82eba...,2023-09-08T11:45:00.000,2023-09-08T18:00:00.000,0,0,8,8,9,0,0,0,3.25,Credit Card,Taxi Affiliation Services,41.97907082,-87.903039661,"{'type': 'Point', 'coordinates': [-87.90303966...",41.899602111,-87.633308037,"{'type': 'Point', 'coordinates': [-87.63330803...",17031980000,17031839100
freq,1,36,377,391,375,2172,4908,5086,973,10247,20661,12689,450,8341,3952,2175,2175,2175,1530,1530,1530,2175,1147


In [8]:
taxi_trips[taxi_trips['fare'].isna()].sample(5)
#Itt a nem érvényes adatokat lehet ignorálni.

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid_location,pickup_census_tract,dropoff_census_tract
321,e00bfaa925338ece77ed73e3e86fc7931e8aa45c,9b6ede7d36f53ba56a1ec8882cc17703224ac426ba9ef9...,2023-09-08T23:15:00.000,2023-09-08T23:30:00.000,870,3.18,8,6,NaN,NaN,NaN,NaN,NaN,Cash,Taxicab Insurance Agency Llc,41.899602111,-87.633308037,"{'type': 'Point', 'coordinates': [-87.63330803...",41.944226601,-87.655998182,"{'type': 'Point', 'coordinates': [-87.65599818...",NaN,NaN
6443,11b7b7a740b84021c4e1a4095926388e7d3c365e,8c76eb82f069c0731a0049cb78898f02cc5ac6990244c9...,2023-09-08T17:15:00.000,2023-09-08T17:30:00.000,1071,3.72,33,8,NaN,NaN,NaN,NaN,NaN,Cash,Sun Taxi,41.849246754,-87.624135298,"{'type': 'Point', 'coordinates': [-87.62413529...",41.891971508,-87.612945414,"{'type': 'Point', 'coordinates': [-87.61294541...",17031841000,17031081402
14349,66dd903352a866a0642e2db3eaf1b040a8e03d7a,15a05a171949e0f432775bd628d0d70a833e9de8fdf048...,2023-09-08T11:15:00.000,2023-09-08T11:15:00.000,NaN,0,76,76,NaN,NaN,NaN,NaN,NaN,Cash,Taxicab Insurance Agency Llc,41.97907082,-87.903039661,"{'type': 'Point', 'coordinates': [-87.90303966...",41.97907082,-87.903039661,"{'type': 'Point', 'coordinates': [-87.90303966...",17031980000,17031980000
9795,628d2f7b441f0aae78442570c95ad5648563efaa,875af0d5f0bf4751ef334d77028966ba1d166cabebd0c3...,2023-09-08T14:45:00.000,2023-09-08T14:45:00.000,170,0.34,8,8,NaN,NaN,NaN,NaN,NaN,Cash,Chicago Independents,41.892507781,-87.626214906,"{'type': 'Point', 'coordinates': [-87.62621490...",41.892507781,-87.626214906,"{'type': 'Point', 'coordinates': [-87.62621490...",17031081500,17031081500
15759,0ec167be921eab498b7da7841b76c0d2e5306165,6b76f128c41cb623a7768f21d1eed44de233cf533eebd0...,2023-09-08T10:15:00.000,2023-09-08T10:15:00.000,NaN,0,8,8,NaN,NaN,NaN,NaN,NaN,Cash,Flash Cab,41.89503345,-87.619710672,"{'type': 'Point', 'coordinates': [-87.61971067...",41.890922026,-87.618868355,"{'type': 'Point', 'coordinates': [-87.61886835...",17031081401,17031081403


Transformations : deal with NaN values

In [9]:
taxi_trips.drop(['pickup_census_tract', 'dropoff_census_tract'], axis=1, inplace=True)
#mert a fele üres

In [10]:
taxi_trips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20799 entries, 0 to 20798
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   trip_id                     20799 non-null  object
 1   taxi_id                     20799 non-null  object
 2   trip_start_timestamp        20799 non-null  object
 3   trip_end_timestamp          20799 non-null  object
 4   trip_seconds                20793 non-null  object
 5   trip_miles                  20799 non-null  object
 6   pickup_community_area       20239 non-null  object
 7   dropoff_community_area      18863 non-null  object
 8   fare                        20749 non-null  object
 9   tips                        20749 non-null  object
 10  tolls                       20749 non-null  object
 11  extras                      20749 non-null  object
 12  trip_total                  20749 non-null  object
 13  payment_type                20799 non-null  ob

In [11]:
#Koordináták duplikációja miatt
taxi_trips.drop(['pickup_centroid_location', 'dropoff_centroid_location'], axis=1, inplace=True)

In [12]:
taxi_trips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20799 entries, 0 to 20798
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   trip_id                     20799 non-null  object
 1   taxi_id                     20799 non-null  object
 2   trip_start_timestamp        20799 non-null  object
 3   trip_end_timestamp          20799 non-null  object
 4   trip_seconds                20793 non-null  object
 5   trip_miles                  20799 non-null  object
 6   pickup_community_area       20239 non-null  object
 7   dropoff_community_area      18863 non-null  object
 8   fare                        20749 non-null  object
 9   tips                        20749 non-null  object
 10  tolls                       20749 non-null  object
 11  extras                      20749 non-null  object
 12  trip_total                  20749 non-null  object
 13  payment_type                20799 non-null  ob

Transformation : renaming community area colunms

In [13]:
taxi_trips.rename(columns={'pickup_community_area' : 'pickup_community_area_id',
                           'dropoff_community_area' : 'dropoff_community_area_id'
                           },inplace=True)

In [14]:
taxi_trips.columns

Index(['trip_id', 'taxi_id', 'trip_start_timestamp', 'trip_end_timestamp',
       'trip_seconds', 'trip_miles', 'pickup_community_area_id',
       'dropoff_community_area_id', 'fare', 'tips', 'tolls', 'extras',
       'trip_total', 'payment_type', 'company', 'pickup_centroid_latitude',
       'pickup_centroid_longitude', 'dropoff_centroid_latitude',
       'dropoff_centroid_longitude'],
      dtype='object')

Transformation : Create helper col for weather

In [15]:
taxi_trips['trip_start_timestamp'] = pd.to_datetime(taxi_trips['trip_start_timestamp'])
taxi_trips['datetime_for_weather'] = taxi_trips['trip_start_timestamp'].dt.floor('H')

In [16]:
taxi_trips['datetime_for_weather']

0       2023-09-08 23:00:00
1       2023-09-08 23:00:00
2       2023-09-08 23:00:00
3       2023-09-08 23:00:00
4       2023-09-08 23:00:00
                ...        
20794   2023-09-08 00:00:00
20795   2023-09-08 00:00:00
20796   2023-09-08 00:00:00
20797   2023-09-08 00:00:00
20798   2023-09-08 00:00:00
Name: datetime_for_weather, Length: 20799, dtype: datetime64[ns]

Transformations : check joining trips to weather

In [17]:
#Get weather data
url = f"https://archive-api.open-meteo.com/v1/era5"
startDate = '2023-09-08'
params = {
    "latitude" : 41.85,
    "longitude" : -87.65,
    "start_date" : startDate,
    "end_date" : startDate,
    "hourly" : "temperature_2m,wind_speed_10m,precipitation,rain"
}


response = requests.get(url, params=params)
weather_data = response.json()

weather_filtered = {
    'datetime': weather_data['hourly']['time'],
    'temperature' : weather_data['hourly']['temperature_2m'],
    'wind' : weather_data['hourly']['wind_speed_10m'],
    'precipitation' : weather_data['hourly']['precipitation'],
    'rain' : weather_data['hourly']['rain']
}
weather_df = pd.DataFrame(weather_filtered)
weather_df['datetime'] = pd.to_datetime(weather_df['datetime'])




In [18]:
weather_df.head()

,datetime,temperature,wind,precipitation,rain
0,2023-09-08 00:00:00,18.8,14.2,0.0,0.0
1,2023-09-08 01:00:00,18.5,13.9,0.0,0.0
2,2023-09-08 02:00:00,18.5,14.1,0.0,0.0
3,2023-09-08 03:00:00,18.6,13.3,0.0,0.0
4,2023-09-08 04:00:00,18.4,17.3,0.0,0.0


In [19]:
#Merging
taxi_trips_with_weather = taxi_trips.merge(weather_df, left_on='datetime_for_weather', right_on='datetime')
taxi_trips_with_weather.drop(['datetime_for_weather', 'datetime'], axis=1, inplace=True)
taxi_trips_with_weather.head()

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area_id,dropoff_community_area_id,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,temperature,wind,precipitation,rain
0,244661871466bd53adbd7e28c6fb95e285ac3511,57fa54b44d1e25d82c1de3200158c2475a894490b1c32c...,2023-09-08 23:45:00,2023-09-09T00:00:00.000,965,4.19,24,32,35,7.1,0,0,42.6,Credit Card,Sun Taxi,41.901206994,-87.676355989,41.878865584,-87.625192142,18.0,20.2,0.0,0.0
1,23f5115ef99a29307567bb35b3faaae761543db7,4a263f78f56815be4acd38658af8fc1824ce4e15f7ec81...,2023-09-08 23:45:00,2023-09-09T00:00:00.000,780,2.7,24,23,10.5,0,0,0,10.5,Cash,Taxi Affiliation Services,41.901206994,-87.676355989,41.900069603,-87.720918238,18.0,20.2,0.0,0.0
2,2376339091c00c12857f09362f5a97d7e4cc4a6f,a9263292f54a406cd0fb2bac808b9aaca91f770ad5e0fd...,2023-09-08 23:45:00,2023-09-08T23:45:00.000,240,0.6,32,32,5,0,0,4,9,Cash,Taxi Affiliation Services,41.877406123,-87.621971652,41.870607372,-87.622172937,18.0,20.2,0.0,0.0
3,1f48cd63505fe83e65f15c64a56abf379409dc85,61d83eb1d34ccd8dac5bdd962bc13e2ac077b0c7ecbf1d...,2023-09-08 23:45:00,2023-09-08T23:45:00.000,420,2.9,76,NaN,17,5.38,0,4,26.88,Credit Card,Globe Taxi,41.97907082,-87.903039661,41.982775009,-87.8773054,18.0,20.2,0.0,0.0
4,217da39f98bd50cb5773d994d34182896f583bf8,694d1417f20bc465bf6f926a2893ffda7512bba4d93021...,2023-09-08 23:45:00,2023-09-09T00:15:00.000,1749,11.97,76,NaN,31.5,0,0,5,37,Credit Card,Taxicab Insurance Agency Llc,41.980264315,-87.913624596,NaN,NaN,18.0,20.2,0.0,0.0


Transformation : data type conversions

In [20]:
taxi_trips_with_weather.head(1)

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area_id,dropoff_community_area_id,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,temperature,wind,precipitation,rain
0,244661871466bd53adbd7e28c6fb95e285ac3511,57fa54b44d1e25d82c1de3200158c2475a894490b1c32c...,2023-09-08 23:45:00,2023-09-09T00:00:00.000,965,4.19,24,32,35,7.1,0,0,42.6,Credit Card,Sun Taxi,41.901206994,-87.676355989,41.878865584,-87.625192142,18.0,20.2,0.0,0.0


In [21]:
#Az AWS nem a df-ből, hanem a csv-ből fog dolgozni. A memória használatot is lehet a megfelelő típus konvertálásokkal csökkenteni.
taxi_trips_with_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20799 entries, 0 to 20798
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   trip_id                     20799 non-null  object        
 1   taxi_id                     20799 non-null  object        
 2   trip_start_timestamp        20799 non-null  datetime64[ns]
 3   trip_end_timestamp          20799 non-null  object        
 4   trip_seconds                20793 non-null  object        
 5   trip_miles                  20799 non-null  object        
 6   pickup_community_area_id    20239 non-null  object        
 7   dropoff_community_area_id   18863 non-null  object        
 8   fare                        20749 non-null  object        
 9   tips                        20749 non-null  object        
 10  tolls                       20749 non-null  object        
 11  extras                      20749 non-null  object    

In [22]:
taxi_trips_with_weather.dropna(inplace=True)

data_types = {
    'trip_end_timestamp' : 'datetime64[ns]',
    'trip_seconds': 'int32',
    'trip_miles': 'float',
    'pickup_community_area_id': 'int8',
    'dropoff_community_area_id': 'int8',
    'fare': 'float',
    'tips': 'float',
    'extras': 'float',
    'trip_total': 'float',

}
taxi_trips_with_weather = taxi_trips_with_weather.astype(data_types)

In [23]:
taxi_trips_with_weather.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18685 entries, 0 to 20798
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   trip_id                     18685 non-null  object        
 1   taxi_id                     18685 non-null  object        
 2   trip_start_timestamp        18685 non-null  datetime64[ns]
 3   trip_end_timestamp          18685 non-null  datetime64[ns]
 4   trip_seconds                18685 non-null  int32         
 5   trip_miles                  18685 non-null  float64       
 6   pickup_community_area_id    18685 non-null  int8          
 7   dropoff_community_area_id   18685 non-null  int8          
 8   fare                        18685 non-null  float64       
 9   tips                        18685 non-null  float64       
 10  tolls                       18685 non-null  object        
 11  extras                      18685 non-null  float64       


Memory usage

In [24]:
taxi_trips.memory_usage(deep=True).sum() / 1024

26759.02734375

In [25]:
taxi_trips_with_weather.memory_usage(deep=True).sum() / 1024

15661.6123046875

Sanity checks

In [26]:
#Megkeresi az értéktartományokat, kiugró értékeket (outliers) 

In [27]:
taxi_trips_with_weather[taxi_trips_with_weather['trip_end_timestamp'] == taxi_trips_with_weather['trip_end_timestamp'].max()]

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area_id,dropoff_community_area_id,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,temperature,wind,precipitation,rain
830,b63045ac1d6ca24563cffa562d9220c73ca0a471,15ebaceb1f5e522414ecc2fb3af8775a525ccbb8427e69...,2023-09-08 22:30:00,2023-09-09 17:00:00,66895,0.0,34,34,50.0,0.0,0,0.0,50.5,Credit Card,5 Star Taxi,41.842076117,-87.633973422,41.842076117,-87.633973422,18.4,21.3,0.0,0.0


In [28]:
taxi_trips_with_weather.nlargest(10, 'fare')

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area_id,dropoff_community_area_id,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,temperature,wind,precipitation,rain
19368,7eabbfbdf27dddc9b7ba2c9dda274e6fe5d53cf6,2f1de337941c9b9db18083ebe41474267c955401ea931e...,2023-09-08 06:30:00,2023-09-08 07:00:00,1680,0.00,8,76,754.0,0.00,0,0.0,754.00,Cash,"Taxicab Insurance Agency, LLC",41.890922026,-87.618868355,41.97907082,-87.903039661,17.9,14.4,1.0,1.0
4045,127ae62d3d93ff5959466dd2f1d1e4e394b81551,c60633c5d2053dd1c2076286b88196cf83f4fe0ecc1945...,2023-09-08 19:00:00,2023-09-08 19:00:00,19,0.00,7,7,385.0,96.38,0,0.0,481.88,Credit Card,Chicago Independents,41.922686284,-87.649488729,41.922686284,-87.649488729,18.4,24.4,0.1,0.1
19980,bec226feb1f9db929925686a6ec4a82563097304,8023557dc87c352423d1acd20c4f6e8390d29f01902c23...,2023-09-08 03:00:00,2023-09-08 05:45:00,9794,142.72,76,74,329.5,0.00,0,12.5,342.00,Cash,Star North Taxi Management Llc,41.980264315,-87.913624596,41.694878966,-87.713192497,18.6,13.3,0.0,0.0
12574,af2bf28364c6a2d166c9d11b2f412b42ada5d958,4d97a6adfef73877de02898cc11d1509ccf607b6e12bc3...,2023-09-08 12:30:00,2023-09-08 12:30:00,4,0.00,12,12,235.0,47.30,0,0.0,283.80,Credit Card,Sun Taxi,42.002015464,-87.774994789,42.002015464,-87.774994789,16.8,18.5,0.3,0.3
12671,613b81fc701d0753a8c28563fcedb6f4919d6fee,4d97a6adfef73877de02898cc11d1509ccf607b6e12bc3...,2023-09-08 12:30:00,2023-09-08 12:30:00,6,0.00,12,12,199.0,50.00,0,0.0,250.00,Credit Card,Sun Taxi,42.002015464,-87.774994789,42.002015464,-87.774994789,16.8,18.5,0.3,0.3
12663,6344901bdda6aab54c92eafea118609759b97eb2,4d97a6adfef73877de02898cc11d1509ccf607b6e12bc3...,2023-09-08 12:30:00,2023-09-08 12:30:00,6,0.00,12,12,198.0,39.80,0,0.0,238.80,Credit Card,Sun Taxi,42.002015464,-87.774994789,42.002015464,-87.774994789,16.8,18.5,0.3,0.3
12761,1ec2649d6df6f0c32cc09c7f53063ef30185d7b4,4d97a6adfef73877de02898cc11d1509ccf607b6e12bc3...,2023-09-08 12:30:00,2023-09-08 12:30:00,7,0.00,12,12,195.0,49.00,0,0.0,245.00,Credit Card,Sun Taxi,42.002015464,-87.774994789,42.002015464,-87.774994789,16.8,18.5,0.3,0.3
12987,87d1b3f49fa877ffe231728937031812beee1842,4d97a6adfef73877de02898cc11d1509ccf607b6e12bc3...,2023-09-08 12:15:00,2023-09-08 12:15:00,4,0.00,12,12,195.0,39.20,0,0.0,235.20,Credit Card,Sun Taxi,41.993930128,-87.758353588,41.993930128,-87.758353588,16.8,18.5,0.3,0.3
13133,115039df582f3fd7c57ad9f0071ea44c5bbe88dc,4d97a6adfef73877de02898cc11d1509ccf607b6e12bc3...,2023-09-08 12:15:00,2023-09-08 12:15:00,25,0.00,12,12,188.0,47.25,0,0.0,236.25,Credit Card,Sun Taxi,41.993930128,-87.758353588,41.993930128,-87.758353588,16.8,18.5,0.3,0.3
176,d44c2f724ddd95bcaa0370d830255fb09bd93924,58b1c31975c36cd531a400d3251e6e31d1ede876dce437...,2023-09-08 23:30:00,2023-09-08 23:30:00,58,0.00,76,76,175.0,0.00,0,0.0,175.00,Cash,Blue Ribbon Taxi Association,41.97907082,-87.903039661,41.97907082,-87.903039661,18.0,20.2,0.0,0.0


In [29]:
print(taxi_trips_with_weather.shape)

(18685, 23)


#### Data Modelling

In [30]:
#Mapping table create
taxi_trips_with_weather['payment_type'].unique()

array(['Credit Card', 'Cash', 'Prcard', 'Mobile', 'Unknown', 'No Charge',
       'Dispute'], dtype=object)

In [31]:
payment_type_master = taxi_trips_with_weather['payment_type'].drop_duplicates().reset_index(drop=True)

In [32]:
for i in range(1, len(payment_type_master) + 1):
    print(i)

1
2
3
4
5
6
7


In [33]:
payment_type_master = pd.DataFrame({
    'payment_type_id': range(1, len(payment_type_master) + 1),
    'payment_type':  payment_type_master
})

In [34]:
payment_type_master

,payment_type_id,payment_type
0,1,Credit Card
1,2,Cash
2,3,Prcard
3,4,Mobile
4,5,Unknown
5,6,No Charge
6,7,Dispute


In [35]:
taxi_trips['company'].unique()

array(['Sun Taxi', 'Taxi Affiliation Services', 'Globe Taxi',
       'Taxicab Insurance Agency Llc', 'City Service', 'Flash Cab',
       'Blue Ribbon Taxi Association', '5 Star Taxi', 'Top Cab',
       'Medallion Leasin', 'Choice Taxi Association',
       'Star North Taxi Management Llc', 'Chicago City Taxi Association',
       'Chicago Independents', 'U Taxicab', 'Koam Taxi Association',
       'Patriot Taxi Dba Peace Taxi Associat',
       'Taxicab Insurance Agency, LLC', '312 Medallion Management Corp',
       'Metro Jet Taxi A.', 'Choice Taxi Association Inc', 'Setare Inc',
       'Taxi Affiliation Services Llc - Yell', '4623 - 27290 Jay Kim',
       'Chicago Taxicab', '2733 - 74600 Benny Jona',
       '6574 - Babylon Express Inc.', '3556 - 36214 RC Andrews Cab',
       '4787 - 56058 Reny Cab Co', '5167 - 71969 5167 Taxi Inc',
       'Petani Cab Corp', '4053 - 40193 Adwar H. Nikola'], dtype=object)

In [36]:
company_type_master = taxi_trips_with_weather['company'].drop_duplicates().reset_index(drop=True)
company_type_master = pd.DataFrame({
    'company_id': range(1, len(company_type_master) + 1),
    'company':  company_type_master
})
company_type_master

,company_id,company
0,1,Sun Taxi
1,2,Taxi Affiliation Services
2,3,Flash Cab
3,4,Blue Ribbon Taxi Association
4,5,5 Star Taxi
5,6,Top Cab
6,7,Choice Taxi Association
7,8,City Service
8,9,Star North Taxi Management Llc
9,10,Chicago City Taxi Association


In [37]:
taxi_trips_with_weather[['payment_type', 'company']]

,payment_type,company
0,Credit Card,Sun Taxi
1,Cash,Taxi Affiliation Services
2,Cash,Taxi Affiliation Services
6,Prcard,Flash Cab
7,Credit Card,Blue Ribbon Taxi Association
...,...,...
20794,Credit Card,Flash Cab
20795,Mobile,Flash Cab
20796,Cash,Taxi Affiliation Services
20797,Cash,Chicago Independents


In [38]:
taxi_trips_with_weather.memory_usage(deep=True).sum() / 1024

15661.6123046875

In [39]:
taxi_trips_with_weather_id = taxi_trips_with_weather.merge(payment_type_master, on='payment_type')

taxi_trips_with_weather_id = taxi_trips_with_weather_id.merge(company_type_master, on='company')

In [40]:
taxi_trips_with_weather_id.sample(5)

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area_id,dropoff_community_area_id,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,temperature,wind,precipitation,rain,payment_type_id,company_id
266,746af80fb4ae9d06a83cb497c0ece8665fdad9d3,d997a0d4cab61cd52db2c82c78aba9a10d73bf469ff400...,2023-09-08 17:00:00,2023-09-08 17:15:00,838,0.81,8,32,8.25,0.00,0,0.0,8.75,Credit Card,Sun Taxi,41.89503345,-87.619710672,41.884987192,-87.620992913,17.8,20.4,3.1,3.1,1,1
13421,ec3a6e4253424d7b93d02f7c19e96c2c1a5b1f66,9054e449dca78c2363d025823dfceb7e33bebc5ca24c92...,2023-09-08 10:00:00,2023-09-08 10:45:00,2812,16.33,76,24,41.25,9.15,0,4.0,54.90,Credit Card,Taxicab Insurance Agency Llc,41.980264315,-87.913624596,41.901206994,-87.676355989,17.4,14.5,0.1,0.1,1,11
11309,7bfbad3ec17e417670377041bbb96364e9dc3543,94cc07a9d2534052a473c0cb2c5cb2be3ffd131ba7db42...,2023-09-08 13:15:00,2023-09-08 14:00:00,2829,17.32,76,24,45.50,11.50,0,11.5,69.00,Credit Card,City Service,41.980264315,-87.913624596,41.901206994,-87.676355989,17.1,17.3,0.6,0.6,1,8
12996,59d436f0fdedc09049bebf1969eee75ad224da8a,d4578429775c32e03fb5139e1d132b91be7e27af9675e7...,2023-09-08 17:15:00,2023-09-08 17:30:00,533,1.32,8,32,7.75,3.00,0,1.0,12.25,Credit Card,Taxicab Insurance Agency Llc,41.891971508,-87.612945414,41.880994471,-87.632746489,17.8,20.4,3.1,3.1,1,11
951,73f1c96cc6e4a6f9003d119d0ccf3339072c3261,35467b44491f6f51eaa0f4fb1cd65e4c23117aa268d9dd...,2023-09-08 20:30:00,2023-09-08 21:30:00,3624,0.49,32,8,8.25,0.00,0,0.0,8.25,Cash,Sun Taxi,41.884987192,-87.620992913,41.890922026,-87.618868355,18.5,24.8,0.3,0.3,2,1


In [41]:
taxi_trips_with_weather_id.drop(['payment_type', 'company'], axis=1, inplace=True)

In [42]:
taxi_trips_with_weather_id.memory_usage(deep=True).sum() / 1024

13272.544921875

In [43]:
payment_type_master.to_csv(data_out+'payment_type_master.csv', index=False)
company_type_master.to_csv(data_out+'company_master.csv', index=False)


#### Checking the modelling

In [44]:
#Ha új payment type van azt frissíteni kell
payment_type_master = taxi_trips_with_weather['payment_type'].drop_duplicates().reset_index(drop=True)
payment_type_master = pd.DataFrame({
    'payment_type_id': range(1, len(payment_type_master) + 1),
    'payment_type':  payment_type_master
})

In [45]:
#Kézzel megcsináljuk a meglévő típusokra a df-et
#Most azt szimuláljuk, hogy van 2 új típus
new_payment_type_data = [
    {'payment_type': 'Credit Card'},
    {'payment_type': 'X'},
    {'payment_type': 'Y'},
]

#Mapping

new_payment_type_mapping = pd.DataFrame(new_payment_type_data)

In [46]:
new_payment_type_mapping

,payment_type
0,Credit Card
1,X
2,Y


In [47]:
#Megnézzük, hogy a payment type masterben melyik a max index.
payment_type_max_id = payment_type_master['payment_type_id'].max()

In [48]:
#Hogyan lehet bővíteni a meglévő payment type-okat újakkal?
#Készítünk az indexnek egy sorozatot, ami a meglévő max értéktől indúl
#a new payment type hossza + payment type max id-ig
range(payment_type_max_id + 1, payment_type_max_id + len(new_payment_type_mapping)+1)

range(8, 11)

In [49]:
#A mapping táblában új oszlop az id-nak
new_payment_type_mapping['payment_type_id'] = range(payment_type_max_id + 1, payment_type_max_id + len(new_payment_type_mapping)+1)

In [50]:
new_payment_type_mapping

,payment_type,payment_type_id
0,Credit Card,8
1,X,9
2,Y,10


In [51]:
#Ellenőrtés, hogy van-e az eredetiban
new_payment_type_mapping['payment_type'].isin(payment_type_master['payment_type'])

0     True
1    False
2    False
Name: payment_type, dtype: bool

In [52]:
#Szűrjük le a mappinget a False-okra
new_payment_type_mapping[~new_payment_type_mapping['payment_type'].isin(payment_type_master['payment_type'])]


,payment_type,payment_type_id
1,X,9
2,Y,10


In [53]:
new_payment_types = new_payment_type_mapping[~new_payment_type_mapping['payment_type'].isin(payment_type_master['payment_type'])]


In [54]:
if not new_payment_types.empty:
    payment_type_master = pd.concat([payment_type_master, new_payment_types],ignore_index=True )

In [55]:
payment_type_master

,payment_type_id,payment_type
0,1,Credit Card
1,2,Cash
2,3,Prcard
3,4,Mobile
4,5,Unknown
5,6,No Charge
6,7,Dispute
7,9,X
8,10,Y
